# Multiclass Text Classification Development

Purpose:
This model predicts a Company's business category based on the text of their homepage website. 

Hypothesis: 
The implicit hypothesis is that websites within each category will use distinctive language that can be used to classify them.

Overall process:
1. Normalize Text (done during eda.ipynb to complete EDA)
2. Label Encoding
3. Feature Extraction (TFIDF & BERT)
4. Model Training
5. Evaulate best performing model and vectorization method

In [35]:
import pandas as pd
import os 
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline

# from transformers import BertTokenizer, BertModel
from transformers import DistilBertTokenizer, DistilBertModel #smaller and faster than BERT
import torch

from sklearn.base import TransformerMixin, BaseEstimator

# read in data

In [2]:
# read data back in from pickle file created with eda.ipynb

# Dynamically get the current working directory
current_dir = os.getcwd()
text_path = os.path.abspath(os.path.join(current_dir, '..', 'output','combined_data.pkl'))

# read data back in 
df_clean = pd.read_pickle(text_path)
df_clean.head()

,Company_ID,CompanyName,Website,Industry,Size_Range,Locality,Country,Current_Employee_Estimate,Total_Employee_Estimate,Category,...,h3,nav_link_text,meta_keywords,meta_description,len_homepage_text,Full_Text,len_Full_Text,clean_text,len_clean_text,clean_text_str
0,99,crinan hotel,crinanhotel.com,hospitality,1 - 10,"ardchonell, argyll and bute, united kingdom",united kingdom,1,3,Corporate Services,...,Accommodation#sep#Activities#sep#Experience Cr...,None,"Crinan hotel, country house hotel, boutique ho...",Crinan Hotel - on waterfront overlooking Loch ...,3467,01546 830261 Crinan · by Lochgilp...,3665,"[crinan, lochgilphead, pa, sr, hotel, history,...",2012,crinan lochgilphead pa sr hotel history ryan f...
1,222,"spot on productions, llc",spotonproductionsllc.com,entertainment,1 - 10,"jackson, mississippi, united states",united states,2,3,"Media, Marketing & Sales",...,None,None,None,"We're Philip Scarborough and Tom Beck, the for...",45,...,75,"[reels, work, storytelling, brought, life, phi...",38,reels work storytelling brought life philip sc...
2,535,akhand jyoti eye hospital,akhandjyoti.in,hospital & health care,11 - 50,"saran, bihar, india",india,8,11,Healthcare,...,Our Girls Help#sep#Donate In Specific Programs...,"why blindness,women empowerment,our impact,abo...",Akhand Jyoti - the largest eye hospital in eas...,"Akhandjyoti, akhand jyoti eye hospital, non-pr...",909,Donate ...,1015,"[donate, gift, someone, sight, support, girl, ...",628,donate gift someone sight support girl child b...
3,642,lasercare eye center,dfweyes.com,medical practice,1 - 10,"irving, texas, united states",united states,4,11,Healthcare,...,None,"home,why choose us,new patient information,pat...",None,Call 214.574.9600 TODAY for an appointment! Th...,1633,...,1820,"[lasik, hotline, main, number, toll, free, irv...",1210,lasik hotline main number toll free irving tx ...
4,675,compumachine inc,compumachine.com,machinery,1 - 10,"danvers, massachusetts, united states",united states,4,9,Industrials,...,None,"home,machines,automation,mastercam,services,ab...",None,Compumachine is proud to offer CNC Machine Too...,192,MACHINES & AUTOMATION HOME MACHINE...,228,"[machines, automation, machines, automation, m...",170,machines automation machines automation master...


# Label Encoding

In [3]:
#Turning the labels into numbers
label_encoder = LabelEncoder()
df_clean['Category_encoded'] = label_encoder.fit_transform(df_clean['Category'])
print(df_clean['Category'].unique())
print(df_clean['Category_encoded'].unique())

['Corporate Services' 'Media, Marketing & Sales' 'Healthcare'
 'Industrials' 'Commercial Services & Supplies' 'Consumer Discretionary'
 'Transportation & Logistics' 'Energy & Utilities' 'Financials'
 'Professional Services' 'Consumer Staples' 'Materials'
 'Information Technology']
[ 3 10  6  7  0  1 12  4  5 11  2  9  8]


# Feature Extraction

I'm going to use k-fold cross-validation to evaluate my models later on. 

In [26]:
# split the data into features (X) and labels (y)
X = df_clean['clean_text_str']
y = df_clean['Category_encoded']

print (X.shape)
print(y.shape)

(71415,)
(71415,)


In [27]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define KFold cross-validator
kf = KFold(n_splits=10, shuffle=True, random_state=42) # using the normal 10 folds

In [6]:
# Define the classification models to be tested
models = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(multi_class='ovr', max_iter=1000),
    'SGD Classifier': SGDClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=100)
}

## Notes:

I've chosen the following models to test: 
1. Naive Bayes
    - Pros:
        - This model is extremely fast and in production can be used as an 'online' model (i.e. can be updated in real time)
        - MultinomialNB is usually very good with discrete features like word counts. 
        - Works well with text data and can hand high-dimiensional data well (low memory usage)
    - Cons
        - Assumes independence (words are not independent in real life)
        - Might be too simplistic --> might not work as well for certain industries 
2. Logistic Regression (on)
    - Pros:
        - Easy to interpret
        - Works well with when the relationship between features and classes is roughly linear (--> frequency of terms correlates with business category)
        - regularization helps with overfitting
    - Cons:
        - requires linear separability
        - sensitive to outliers
        - not the best with a large number of features (works best with small or medium-sized datasets)
3. Stochastic Gradient Descent (SGD)
    - Pros:
        - Can scale well for large datasets
        - supports regularlization for overfitting
        - Can learn incrementally in streams or batches
        - can be used with different loss functions
    - Cons:
        - requires careful hyperparameter tuning 
        - might require a lot of iterations
        - can be instable
        - not as interpretable as other algorithms

4. Random Forest
    - Pros:
        - robust to overfitting
        - can handle non-linear relationships
        - gives some insight into which features to select
        - tolerates missing data and outliers
    - Cons:
        - slow and computationally expensive
        - not as interpretable
        - can struggle with text data or data with high-dimensionality (if chosen would benefit from dimensionality reduction)


## DistilBERT embeddings

Because DistilBERT is bidirectional, it accounts for the context of words while TF-IDF does not. BERT uses deep neural networks so it will be
more computationally expensive than TF-IDF, but it's still worth testing. I'm using the DistilBERT model over BERT because I'm not working with GPU. 

In [39]:
class DistilBERTEmbeddingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model_name='distilbert-base-uncased', max_length=512): #use distilbert because it's smaller and faster than BERT
        self.model_name = model_name  # Explicitly set model_name
        self.tokenizer = DistilBertTokenizer.from_pretrained(self.model_name)
        self.model = DistilBertModel.from_pretrained(self.model_name)
        self.max_length = max_length

    def fit(self, X, y=None):
        # Fit method required for scikit-learn compatibility
        return self

    def transform(self, X):
        embeddings = []
        for text in X:
            # Tokenize and convert text to tensors
            tokens = self.tokenizer(text, return_tensors='pt', padding='max_length',
                                    truncation=True, max_length=self.max_length)
            
            # Ensure no gradient computation for embeddings
            with torch.no_grad():
                output = self.model(**tokens)
                # Take the CLS token embedding
                cls_embedding = output.last_hidden_state[:, 0, :].numpy()
            
            embeddings.append(cls_embedding[0])  # Append as numpy array
            
        return np.array(embeddings)


In [40]:
# Precompute BERT embeddings for X_train and X_test

# instead of recalculating the BERT embeddings during cross-validation, precompute them
# once and store them 
# this should greatly reduce time for model training and evaluation
bert_transformer = DistilBERTEmbeddingTransformer()
X_train_embeddings = bert_transformer.transform(X_train)
X_test_embeddings = bert_transformer.transform(X_test)


c:\Users\megan\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:
# Create a sample pipeline with BERT embeddings and a classifier (RandomForest in this case)
bert_pipeline = Pipeline([
    ('bert', BERTEmbeddingTransformer()),  # Use BERT embeddings
    ('classifier', RandomForestClassifier())  # Random forest classifier
])

# Example training data (text)
X_train = ['This is a text sample.', 'Another example text.', 'Yet another sample of text data.']
y_train = [0, 1, 0]

# Example: Cross-validation using BERT embeddings and the classifier
scores = cross_val_score(bert_pipeline, X_train, y_train, cv=2)
print(f"Cross-validation accuracy: {scores.mean():.4f}")

c:\Users\megan\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\megan\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(("The least populated class in y has only %d"
c:\Users\megan\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issue

Cross-validation accuracy: 0.7500


# Model Evaluation

In [8]:
# Initialize lists to store results for comparison
results = []

# Iterate over the models and compare pipelines (TF-IDF vs BERT embeddings)
for model_name, model in models.items():
    print(f"\n=== {model_name} ===")
    ### TF-IDF Pipeline
    tfidf_model_pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=100, stop_words='english')),
        (model_name, model)
    ])

    # Cross-Validation for TF-IDF
    tfidf_scores = cross_val_score(tfidf_model_pipeline, X_train, y_train, cv=kf, scoring='accuracy')
    
    # Train on TF-IDF
    tfidf_model_pipeline.fit(X_train, y_train)
    y_pred_tfidf = tfidf_model_pipeline.predict(X_test)
    tfidf_accuracy = accuracy_score(y_test, y_pred_tfidf)
    tfidf_report = classification_report(y_test, y_pred_tfidf, target_names=label_encoder.classes_, output_dict=True)

    print(f"TF-IDF {model_name} Confusion Matrix of Category Performance: {tfidf_accuracy}")
    print(tfidf_report)

    # Store TF-IDF results
    results.append({
        'Model': model_name,
        'Pipeline': 'TF-IDF',
        'Cross_Val_Accuracy': tfidf_scores.mean(),
        'Test_Accuracy': tfidf_accuracy,
        'Precision': tfidf_report['weighted avg']['precision'],
        'Recall': tfidf_report['weighted avg']['recall'],
        'F1-Score': tfidf_report['weighted avg']['f1-score']
    })

# Create DataFrame for all the results
results_df = pd.DataFrame(results)

# Display results for comparison
print(results_df)



=== Naive Bayes ===
TF-IDF Naive Bayes Confusion Matrix of Category Performance: 0.6066744457409569
{'Commercial Services & Supplies': {'precision': 0.5278044103547459, 'recall': 0.6019682886823401, 'f1-score': 0.5624521072796935, 'support': 1829}, 'Consumer Discretionary': {'precision': 0.6056338028169014, 'recall': 0.05269607843137255, 'f1-score': 0.09695603156708003, 'support': 816}, 'Consumer Staples': {'precision': 0.4980750721847931, 'recall': 0.6699029126213593, 'f1-score': 0.5713497101849296, 'support': 1545}, 'Corporate Services': {'precision': 0.5210270030987163, 'recall': 0.563967417345472, 'f1-score': 0.5416474919466175, 'support': 2087}, 'Energy & Utilities': {'precision': 0.8159879336349924, 'recall': 0.6605616605616605, 'f1-score': 0.7300944669365721, 'support': 1638}, 'Financials': {'precision': 0.8160088938299055, 'recall': 0.729985082048732, 'f1-score': 0.7706036745406825, 'support': 2011}, 'Healthcare': {'precision': 0.6926458832933653, 'recall': 0.8335738335738335,

KeyboardInterrupt: 

In [10]:
results_df = pd.DataFrame(results)
results_df

,Model,Pipeline,Cross_Val_Accuracy,Test_Accuracy,Precision,Recall,F1-Score
0,Naive Bayes,TF-IDF,0.609002,0.606674,0.617974,0.606674,0.593908
1,Logistic Regression,TF-IDF,0.643709,0.643081,0.639901,0.643081,0.638863
2,SGD Classifier,TF-IDF,0.610522,0.608681,0.594312,0.608681,0.595254


In [24]:
for model_name, model in models.items():
    print(f"\n=== {model_name} ===")


=== Naive Bayes ===

=== Logistic Regression ===

=== SGD Classifier ===

=== Random Forest ===


In [30]:
# Store BERT results
bert_results = []

# Iterate over models using the BERT pipeline
for model_name, model in models.items():  # Changed loop variable to 'model_name'
    print(f"\n=== {model_name} ===")
    # BERT Pipeline
    bert_model_pipeline = Pipeline([
        ('bert', BERTEmbeddingTransformer()),  # BERT transformer
        ('classifier', model)  # 'classifier' is a generic name for the model
    ])
    
    # Cross-Validation for BERT
    bert_scores = cross_val_score(bert_model_pipeline, X_train, y_train, cv=kf, scoring='accuracy')
    
    # Train on BERT
    bert_model_pipeline.fit(X_train, y_train)
    y_pred_bert = bert_model_pipeline.predict(X_test)
    
    # Calculate accuracy and classification report
    bert_accuracy = accuracy_score(y_test, y_pred_bert)
    bert_report = classification_report(y_test, y_pred_bert, target_names=label_encoder.classes_, output_dict=True)
    
    # Append BERT results to the results list
    bert_results.append({
        'Model': model_name,  # Changed to use 'model_name' from the loop
        'Cross_Val_Accuracy': bert_scores.mean(),
        'Test_Accuracy': bert_accuracy,
        'Precision': bert_report['weighted avg']['precision'],
        'Recall': bert_report['weighted avg']['recall'],
        'F1-Score': bert_report['weighted avg']['f1-score']
    })

# Convert BERT results to DataFrame
bert_results_df = pd.DataFrame(bert_results)
print("BERT Results:")
print(bert_results_df)



=== Naive Bayes ===


c:\Users\megan\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


KeyboardInterrupt: 

In [10]:
# # initialize list to store results for comparison
# results = []
# # Iterate over the models and compare pipelines (TF-IDF vs BERT embeddings)
# for model_name, model in models.items():
#     print(f"\n=== {model_name} ===")
    
#     ### TF-IDF Pipeline
#     tfidf_model_pipeline = Pipeline([
#         ('tfidf', TfidfVectorizer(max_features=1000, stop_words='english')),
#         (model_name, model)
#     ])

#     # Cross-Validation for TF-IDF
#     tfidf_scores = cross_val_score(tfidf_model_pipeline, X_train, y_train, cv=kf, scoring='accuracy')
#     print(f"TF-IDF {model_name} Cross-Validation Accuracy: {tfidf_scores.mean():.4f}")
    
#     # Train on TF-IDF
#     tfidf_model_pipeline.fit(X_train, y_train)
#     y_pred_tfidf = tfidf_model_pipeline.predict(X_test)
#     print(f"TF-IDF {model_name} Confusion Matrix of Category Performance:")
#     print(classification_report(y_test, y_pred_tfidf, target_names=label_encoder.classes_))

#     ### BERT Pipeline
#     bert_model_pipeline = Pipeline([
#         ('bert', BERTEmbeddingTransformer()),  # BERT Embeddings
#         (model_name, model)
#     ])

#       # Cross-Validation for BERT
#     bert_scores = cross_val_score(bert_model_pipeline, X_train, y_train, cv=kf, scoring='accuracy')
#     print(f"BERT {model_name} Cross-Validation Accuracy: {bert_scores.mean():.4f}")
    
#     # Train on BERT
#     bert_model_pipeline.fit(X_train, y_train)
#     y_pred_bert = bert_model_pipeline.predict(X_test)
#     print(f"BERT {model_name} Confusion Matrix of Category Performance:")
#     print(classification_report(y_test, y_pred_bert, target_names=label_encoder.classes_))



=== Naive Bayes ===
TF-IDF Naive Bayes Cross-Validation Accuracy: 0.8085
TF-IDF Naive Bayes Confusion Matrix of Category Performance:
                                precision    recall  f1-score   support

Commercial Services & Supplies       0.76      0.82      0.79      1829
        Consumer Discretionary       0.66      0.51      0.58       816
              Consumer Staples       0.72      0.88      0.79      1545
            Corporate Services       0.82      0.79      0.80      2087
            Energy & Utilities       0.87      0.82      0.84      1638
                    Financials       0.93      0.89      0.91      2011
                    Healthcare       0.91      0.88      0.89      2079
                   Industrials       0.75      0.77      0.76      1015
        Information Technology       0.71      0.84      0.77      1804
                     Materials       0.79      0.67      0.72       798
      Media, Marketing & Sales       0.78      0.83      0.81      1840


c:\Users\megan\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


AttributeError: 'BERTEmbeddingTransformer' object has no attribute 'model_name'